In [1]:
from poem.instance_creation_factories.triples_factory import TriplesFactory
from poem.models.unimodal.complex_cwa import ComplexCWA
from torch import optim
import numpy as np
from poem.training.cwa import CWATrainingLoop
import os
import logging
import sys
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stderr)
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

**Step 1: Create instances**

In [2]:
path_to_training_data = '../tests/resources/test.txt'

In [3]:
import timeit

In [4]:
factory = TriplesFactory(path=path_to_training_data)

**Step 2: Configure KGE model**

In [5]:
kge_model = ComplexCWA(triples_factory=factory,
                       embedding_dim=200,
                       input_dropout=0.2
                      )

**Step 3: Configure Training Loop**

In [6]:
optimizer = optim.Adam(params=kge_model.get_grad_params())
cwa_training_loop = CWATrainingLoop(model=kge_model, optimizer=optimizer)

**Step 4: Train**

In [7]:
losses = cwa_training_loop.train(num_epochs=5,
                                 batch_size=256)

Training epoch on cuda: 100%|██████████| 5/5 [00:20<00:00,  4.17s/it]


In [8]:
losses

[0.6931468626471745,
 0.6721640842141317,
 0.2619327757018129,
 0.059521213734526256,
 0.026114851929317878]

In [9]:
from poem.evaluation import RankBasedEvaluator

### Evaluation without filtering triples

In [10]:
evaluator = RankBasedEvaluator(kge_model, filter_neg_triples=False)

In [11]:
test_triples = factory.map_triples_to_id(path_to_triples=path_to_training_data)

In [12]:
evaluator.evaluate(test_triples, batch_size=4096)

⚡️ Evaluating triples : 100%|██████████| 59.1k/59.1k [00:00<00:00, 75.7ktriple(s)/s]


MetricResults(mean_rank=5184.070956983968, mean_reciprocal_rank=9.31082934096257e-05, adjusted_mean_rank=0.7632052302360535, adjusted_mean_reciprocal_rank=5.181576728820801, hits_at_k={1: 9.31082934096257e-05, 3: 0.00017775219650928545, 5: 0.0002793248802288771, 10: 0.00060943610231755})

### Evaluation with filtering triples

In [13]:
evaluator = RankBasedEvaluator(kge_model, filter_neg_triples=True)

In [14]:
test_triples = factory.map_triples_to_id(path_to_triples=path_to_training_data)

In [15]:
# Since filtering the triples requires to load all triples into memory, the batch size has to be reduced.
evaluator.evaluate(test_triples, batch_size=1024)

⚡️ Evaluating triples : 100%|██████████| 59.1k/59.1k [00:05<00:00, 10.1ktriple(s)/s]


MetricResults(mean_rank=5175.177125831627, mean_reciprocal_rank=9.31082934096257e-05, adjusted_mean_rank=0.7618958950042725, adjusted_mean_reciprocal_rank=5.194695472717285, hits_at_k={1: 9.31082934096257e-05, 3: 0.00017775219650928545, 5: 0.0002793248802288771, 10: 0.00060943610231755})